<a href="https://colab.research.google.com/github/unt-iialab/medical-concept-normalization/blob/master/medical-concept-normalization/fine-tuning-experiments/askpatient/pubmed_twadr_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

In [2]:
base_dir = 'medical-concept-normalization'

In [3]:
os.mkdir(base_dir)

In [4]:
os.mkdir('medical-concept-normalization/data_collection')

**Install fastai and clone the fastai for text classification**



In [5]:
!curl https://course-v3.fast.ai/setup/colab | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0curl: (6) Could not resolve host: course-v3.fast.ai


In [6]:
!git clone https://github.com/wshuyi/demo-nlp-classification-fastai.git

Cloning into 'demo-nlp-classification-fastai'...
remote: Enumerating objects: 9, done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 9
Unpacking objects: 100% (9/9), done.


In [7]:
from fastai import *
from fastai.text import *

**Set the path of the data collections**

In [8]:
path = Path('medical-concept-normalization/data_collection')

In [10]:
# training data set

train_csv = path/'TwADR-L.fold-4.train.csv'
train = pd.read_csv(train_csv, header=None)
#train.head()
#train.info()

In [11]:
# validating data set

valid = pd.read_csv(path/"TwADR-L.fold-4.validation.csv",header=None)
valid.head()

,0,1
0,61,very stupid
1,74,useless without
2,93,shooting
3,195,makes me so damn bloated
4,231,gives me man vivid dreams


In [12]:
# testing data set

test = pd.read_csv(path/"TwADR-L.fold-4.test.csv",header=None)
test.head()

,0,1
0,13,insomnia
1,13,insomnia
2,56,painful b..
3,62,hypomanic
4,84,paraplegic


In [13]:
# Creating DataBunch for language modelling

data_lm = TextLMDataBunch.from_csv(path, 'TwADR_pubmed.csv')

In [14]:
# Creating DataBunch for classification

# data_clas = TextClasDataBunch.from_df(path, train, valid, test, vocab=data_lm.train_ds.vocab, bs=128)

In [15]:
data_clas = TextClasDataBunch.from_df(path, train, valid, vocab=data_lm.train_ds.vocab, bs=128)

In [16]:
data_clas.add_test(test,label=0)


In [17]:
data_lm.save()
data_clas.save()

In [18]:
data_lm.train_ds.vocab.itos

['xxunk',
 'xxpad',
 'xxbos',
 'xxeos',
 'xxfld',
 'xxmaj',
 'xxup',
 'xxrep',
 'xxwrep',
 'the',
 'of',
 'and',
 'in',
 'to',
 'a',
 'diclofenac',
 'was',
 'with',
 'were',
 'for',
 'pain',
 'sodium',
 'drug',
 'by',
 'on',
 'patients',
 'as',
 'that',
 'is',
 'at',
 'or',
 'mg',
 'increased',
 'group',
 'from',
 'mood',
 'study',
 'this',
 'treatment',
 'after',
 'p',
 'release',
 'an',
 'be',
 ',',
 'me',
 'disorder',
 'effect',
 'not',
 'abnormal',
 'i',
 'withdrawal',
 'decreased',
 'than',
 'weight',
 'anxiety',
 'compared',
 'using',
 'groups',
 'are',
 'no',
 'effects',
 'my',
 'both',
 'symptoms',
 'two',
 'inflammatory',
 'significantly',
 'acute',
 'potassium',
 'drugs',
 'significant',
 'h',
 'between',
 'anti',
 'day',
 'these',
 'skin',
 'time',
 'voltaren',
 'disorders',
 'used',
 'all',
 'syndrome',
 'dose',
 'blood',
 'behavior',
 'which',
 'results',
 '-',
 '(',
 ')',
 'placebo',
 'it',
 'efficacy',
 'had',
 'sleeplessness',
 'concentration',
 'oral',
 'ds',
 'have',


In [19]:
model_path = path/'models'
model_path.mkdir(exist_ok=True)
url = 'http://files.fast.ai/models/wt103_v1/'
download_url(f'{url}lstm_wt103.pth', model_path/'lstm_wt103.pth')
download_url(f'{url}itos_wt103.pkl', model_path/'itos_wt103.pkl')

In [20]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

In [21]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.842000,4.177683,0.351494,15:46


In [22]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.043366,3.786264,0.386435,15:38


In [23]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.733979,3.738708,0.391415,15:40


In [24]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.690859,3.706515,0.394557,15:47


In [25]:
learn.save_encoder('TwADR_pubmed_fold4_first')

In [26]:
learn = text_classifier_learner(data_clas,AWD_LSTM, drop_mult=0.5)
learn.load_encoder('TwADR_pubmed_fold4_first')
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,7.205054,6.546072,0.091667,00:25


In [27]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,6.250976,5.724273,0.150000,00:24


In [28]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,5.745228,5.131246,0.166667,01:04


In [29]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,5.438085,4.706786,0.175000,01:04
1,5.226608,4.589098,0.225000,01:01


In [30]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,4.986502,4.321394,0.258333,01:01
1,4.807708,4.221321,0.258333,01:02


In [31]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,4.572659,4.142347,0.275000,00:59
1,4.416861,3.966407,0.325000,00:59


In [32]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,4.214502,3.840574,0.316667,01:04
1,4.070415,3.812047,0.283333,01:02


In [33]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.902004,3.727372,0.366667,01:08
1,3.760525,3.700618,0.341667,01:03


In [34]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.569719,3.637098,0.333333,01:00
1,3.493597,3.597358,0.308333,01:00
2,3.322882,3.514940,0.408333,01:02
3,3.158712,3.516770,0.400000,01:03


In [35]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.997350,3.506801,0.408333,00:59
1,2.921129,3.485883,0.366667,01:02
2,2.732234,3.425692,0.433333,01:04
3,2.564222,3.387824,0.433333,01:02


In [36]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.448110,3.482567,0.400000,00:58
1,2.372349,3.558305,0.358333,01:00
2,2.207558,3.426339,0.375000,01:01
3,2.090102,3.461089,0.350000,00:59


In [37]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.941489,3.386345,0.391667,01:03
1,1.904890,3.454435,0.400000,01:04
2,1.784073,3.499000,0.408333,01:01
3,1.628874,3.442484,0.425000,01:01


In [38]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.510908,3.480734,0.400000,01:03
1,1.471902,3.598947,0.375000,01:04
2,1.371918,3.448247,0.400000,01:03
3,1.239963,3.489529,0.400000,01:06


In [39]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.142132,3.510531,0.383333,01:00
1,1.169466,3.595794,0.366667,01:01
2,1.085663,3.586222,0.375000,00:59
3,0.979054,3.551101,0.391667,01:03


In [40]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.914146,3.554577,0.400000,00:59
1,0.992914,3.738359,0.400000,01:01
2,0.905616,3.723634,0.358333,01:04
3,0.822702,3.732988,0.358333,01:03


In [41]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.750544,3.731293,0.400000,01:01
1,0.716185,3.672532,0.400000,00:59


In [42]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.721606,3.981314,0.350000,01:03
1,0.693167,3.766038,0.383333,01:01


In [43]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.650406,3.755312,0.391667,01:04


In [44]:
# save the best model

learn.save_encoder('TwADR_pubmed_fold4')

In [45]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.627051,3.853754,0.366667,01:07


In [46]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.617981,3.923063,0.408333,01:05


In [47]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.609477,3.832782,0.425000,01:02


# Part three: Predict on the test dataset

In [48]:
# use the test data for prediction

preds, y = learn.get_preds(DatasetType.Test)

In [49]:
predictions, *_ = learn.get_preds(DatasetType.Test)
labels = np.argmax(predictions, 1)
predict_list = labels.tolist()
standard_list = test[0].tolist()

print(len(predict_list))
print(len(standard_list))

print(predict_list)
print(standard_list)

143
143
[1787, 1787, 56, 791, 791, 109, 852, 189, 189, 230, 231, 1936, 233, 233, 1787, 393, 1136, 2063, 982, 819, 2002, 187, 561, 596, 1290, 1432, 749, 749, 97, 1597, 792, 1974, 1458, 1458, 2147, 824, 908, 231, 923, 792, 791, 791, 974, 978, 982, 978, 978, 978, 978, 1546, 1828, 1089, 1815, 1099, 1877, 2147, 1669, 1136, 1136, 1145, 2002, 2002, 1340, 1184, 1184, 1718, 1204, 978, 24, 1265, 857, 582, 1308, 1227, 2149, 852, 1340, 1936, 2002, 1508, 1508, 1511, 1526, 1529, 1711, 1682, 1594, 1595, 1597, 1597, 1597, 2168, 1526, 791, 791, 791, 1610, 978, 1073, 1669, 2081, 1680, 1701, 1526, 1594, 56, 1767, 1767, 1669, 1679, 1767, 1815, 1578, 2191, 1787, 1048, 2023, 1857, 749, 825, 2168, 1936, 1936, 1942, 2002, 1899, 791, 2191, 1787, 189, 1508, 224, 791, 2103, 2014, 2119, 2147, 1319, 1787, 2147, 1936, 1349, 2149]
[13, 13, 56, 62, 84, 109, 189, 189, 189, 230, 231, 233, 233, 233, 393, 393, 396, 418, 418, 452, 452, 483, 561, 596, 712, 729, 749, 749, 750, 791, 792, 792, 809, 809, 813, 824, 908, 908, 92

In [50]:
correct_num = 0

for i in range(0, len(standard_list)):
  if predict_list[i] == standard_list[i]:
    correct_num = correct_num+1
  else:
    pass
print(correct_num)

predict_accuracy = correct_num/len(predict_list)
print(predict_accuracy)

59
0.4125874125874126
